In [ ]:
'''
Capstone Project: CIND820 
Harpreet Kang
June 7, 2024
'''

In [11]:
%matplotlib inline
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV,KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats
import statsmodels.api as sm
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB


# Loading the libraries

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Displaying all the columns and rows

In [16]:
data = pd.read_excel('dataset_lfs_2024.xlsx')
# Loading the data

In [17]:
data.dropna(subset=["HRLYEARN"],inplace=True)
# Removing the null values in HRLYEARN column
data=data[data['MJH']==1]
# Filtering for single job holders
data=data[data['SCHOOLN']==1]
# Filtering for non students
data=data[data['FTPTMAIN']==1]
# Filtering for full-time workers
data=data[data['PERMTEMP']==1]
# Filtering for permanent workers
data['HRLYEARN']=data['HRLYEARN']/100
# The data dictionary for this dataset indicated that the last 2 values of this numeric column were the decimal points. So dividing the HRLYEARN column by 100 will add 2 decimal points. 
data['UHRSMAIN']=data['UHRSMAIN']/10
data['AHRSMAIN']=data['AHRSMAIN']/10
data['UTOTHRS']=data['UTOTHRS']/10
data['ATOTHRS']=data['ATOTHRS']/10
data['HRSAWAY']=data['HRSAWAY']/10
data['PAIDOT']=data['PAIDOT']/10
data['UNPAIDOT']=data['UNPAIDOT']/10
data['XTRAHRS']=data['XTRAHRS']/10
# # The data dictionary for this dataset indicated that the last 1 values of the 8 numeric columns above were the decimal points. So dividing the UHRSMAIN column by 100 will add 1 decimal points.

In [ ]:
#profile1 = ProfileReport(data, title="Profiling Report")
#profile1.to_file("EDA after eliminating blanks of Usual Hourly Wages (HRLYEARN).html")
# Generating the profiling report after removing the null values in HRLYEARN column

In [ ]:
#caclulating the high imbalances for variables: LFSSTAT ,MJH, FTPTMAIN, PERMTEMP, SCHOOLN ,HRSAWAY, PAIDOT, UNPAIDOT, XTRAHRS

#for column in ['LFSSTAT','MJH','FTPTMAIN','PERMTEMP','SCHOOLN','HRSAWAY','PAIDOT','UNPAIDOT','XTRAHRS']:
    #print(data[column].value_counts(normalize=True)*100)   

In [18]:
full_variable_name_mapping={
'rec_num':'Order of record in file',
'survyear':'Survey year',
'survmnth':'Survey month',
'lfsstat':'Labour force status',
'prov':'Province',
'cma':'Nine largest CMAs',
'age_12':'Five-year age group of respondent',
'age_6':'Age in 2 and 3 year groups, 15 to 29',
'sex':'Sex of respondent',
'marstat':'Marital status of respondent',
'educ':'Highest educational attainment',
'mjh':'Single or multiple jobholder',
'everwork':'Identifies if a person has worked in the last year',
'ftptlast':'Full- or part-time status of last job',
'cowmain':'Class of worker, main job',
'immig':'Immigrant status',
'naics_21':'Industry of main job',
'noc_10':'Occupation at main job (noc_10)',
'noc_43':'Occupation at main job (noc_43)',
'yabsent':'Reason of absence, full week',
'wksaway':'Number of weeks absent from work',
'payaway':'Paid for time off, full-week absence only',
'uhrsmain':'Usual hours worked per week at main job',
'ahrsmain':'Actual hours worked per week at main job',
'ftptmain':'Full- or part-time status at main or only job',
'utothrs':'Usual hours worked per week at all jobs',
'atothrs':'Actual hours worked per week at all jobs',
'hrsaway':'Hours away from work, part-week absence only',
'yaway':'Reason for part-week absence',
'paidot':'Paid overtime hours in reference week',
'unpaidot':'Unpaid overtime hours in reference week',
'xtrahrs':'Number of overtime or extra hours worked',
'whypt':'Reason for part-time work',
'tenure':'Job tenure with current employer',
'prevten':'Job tenure with previous employer',
'hrlyearn':'Usual hourly wages',
'union':'Union status',
'permtemp':'Job permanency',
'estsize':'Establishment size',
'firmsize':'Firm size',
'durunemp':'Duration of unemployment',
'flowunem':'Flows into unemployment',
'unemftpt':'Job seekers by type of work sought and temporary layoffs by work status of last job',
'whylefto':'Reason for leaving job during previous year (whylefto)',
'whyleftn':'Reason for leaving job during previous year (whyleftn)',
'durjless':'Duration of joblessness',
'availabl':'Availability during the reference week',
'lkpubag':'Unemployed, used public employment agency',
'lkemploy':'Unemployed, checked with employers directly',
'lkrels':'Unemployed, checked with friends or relatives',
'lkatads':'Unemployed, looked at job ads',
'lkansads':'Unemployed, placed or answered ads',
'lkothern':'Unemployed, other methods',
'prioract':'Main activity before started looking for work',
'ynolook':'Reason for not looking for work during the reference week',
'tlolook':'Temporary layoff, looked for work during the last four weeks',
'schooln':'Current student status',
'efamtype':'Type of economic family',
'agyownk':'Age of youngest child',
'finalwt':'Standard final weight'}

# Mapping descriptions to the variable names

In [ ]:
#data.describe()
# Displaying the summary statistics of the data

In [ ]:
#data.info()
# Displaying the data types of the columns

In [19]:
data_missing_values= data.isnull().sum()
# Counting the missing values in each column
data_percent_missing = round(data.isnull().sum() * 100 / len(data),2)
# Calculating the percentage of missing values in each column
missing_values_percent=pd.DataFrame(data_percent_missing, columns=['Missing Values %'])
# Creating a dataframe to display the missing values percentage
missing_values_percent=missing_values_percent.rename_axis('Variables')
# Renaming the index

In [ ]:
#data.head()
# Displaying the first 5 rows of the data

In [20]:
def statistics_table(data_frame):
    data_type_mapping={
    'rec_num':'Nominal',
    'survyear':'Ordinal',
    'survmnth':'Ordinal',
    'lfsstat':'Nominal',
    'prov':'Nominal',
    'cma':'Nominal',
    'age_12':'Ordinal',
    'age_6':'Ordinal',
    'sex':'Nominal',
    'marstat':'Nominal',
    'educ':'Ordinal',
    'mjh':'Nominal',
    'everwork':'Nominal',
    'ftptlast':'Nominal',
    'cowmain':'Nominal',
    'immig':'Nominal',
    'naics_21':'Nominal',
    'noc_10':'Nominal',
    'noc_43':'Nominal',
    'yabsent':'Nominal',
    'wksaway':'Nominal',
    'payaway':'Nominal',
    'uhrsmain':'Continuous',
    'ahrsmain':'Continuous',
    'ftptmain':'Nominal',
    'utothrs':'Continuous',
    'atothrs':'Continuous',
    'hrsaway':'Continuous',
    'yaway':'Nominal',
    'paidot':'Continuous',
    'unpaidot':'Continuous',
    'xtrahrs':'Continuous',
    'whypt':'Nominal',
    'tenure':'Discrete',
    'prevten':'Discrete',
    'hrlyearn':'Continuous',
    'union':'Nominal',
    'permtemp':'Nominal',
    'estsize':'Ordinal',
    'firmsize':'Ordinal',
    'durunemp':'Discrete',
    'flowunem':'Nominal',
    'unemftpt':'Nominal',
    'whylefto':'Nominal',
    'whyleftn':'Nominal',
    'durjless':'Discrete',
    'availabl':'Nominal',
    'lkpubag':'Nominal',
    'lkemploy':'Nominal',
    'lkrels':'Nominal',
    'lkatads':'Nominal',
    'lkansads':'Nominal',
    'lkothern':'Nominal',
    'prioract':'Nominal',
    'ynolook':'Nominal',
    'tlolook':'Nominal',
    'schooln':'Nominal',
    'efamtype':'Nominal',
    'agyownk':'Ordinal',
    'finalwt':'Continuous'}

    data_object_mapping={
    'rec_num':'Qualitative',
    'survyear':'Qualitative',
    'survmnth':'Qualitative',
    'lfsstat':'Qualitative',
    'prov':'Qualitative',
    'cma':'Qualitative',
    'age_12':'Qualitative',
    'age_6':'Qualitative',
    'sex':'Qualitative',
    'marstat':'Qualitative',
    'educ':'Qualitative',
    'mjh':'Qualitative',
    'everwork':'Qualitative',
    'ftptlast':'Qualitative',
    'cowmain':'Qualitative',
    'immig':'Qualitative',
    'naics_21':'Qualitative',
    'noc_10':'Qualitative',
    'noc_43':'Qualitative',
    'yabsent':'Qualitative',
    'wksaway':'Qualitative',
    'payaway':'Qualitative',
    'uhrsmain':'Quantitative',
    'ahrsmain':'Quantitative',
    'ftptmain':'Qualitative',
    'utothrs':'Quantitative',
    'atothrs':'Quantitative',
    'hrsaway':'Quantitative',
    'yaway':'Qualitative',
    'paidot':'Quantitative',
    'unpaidot':'Quantitative',
    'xtrahrs':'Quantitative',
    'whypt':'Qualitative',
    'tenure':'Quantitative',
    'prevten':'Quantitative',
    'hrlyearn':'Quantitative',
    'union':'Qualitative',
    'permtemp':'Qualitative',
    'estsize':'Qualitative',
    'firmsize':'Qualitative',
    'durunemp':'Qualitative',
    'flowunem':'Qualitative',
    'unemftpt':'Qualitative',
    'whylefto':'Qualitative',
    'whyleftn':'Qualitative',
    'durjless':'Qualitative',
    'availabl':'Qualitative',
    'lkpubag':'Qualitative',
    'lkemploy':'Qualitative',
    'lkrels':'Qualitative',
    'lkatads':'Qualitative',
    'lkansads':'Qualitative',
    'lkothern':'Qualitative',
    'prioract':'Qualitative',
    'ynolook':'Qualitative',
    'tlolook':'Qualitative',
    'schooln':'Qualitative',
    'efamtype':'Qualitative',
    'agyownk':'Qualitative',
    'finalwt':'Quantitative'}
    data_table=data_frame.columns.to_frame(index=False)
    data_table=data_table.rename(columns={0:'Variable Name'})
    columns=list(data_table.iloc[:,0] )
    data_table_mean_lst=[]
    data_table_median_lst=[]
    data_table_min_lst=[]
    data_table_max_lst=[]
    data_table_std_lst=[]
    data_table_object_lst=[]
    data_table_dtype_lst=[]

    for column in columns:
        data_table_mean_lst.append(round(data_frame[column].mean(),2))
        data_table_median_lst.append(round(data_frame[column].median(),2))
        data_table_min_lst.append(round(data_frame[column].min(),2))
        data_table_max_lst.append(round(data_frame[column].max(),2))
        data_table_std_lst.append(round(data_frame[column].std(),2))
        data_table_object_lst.append(data_object_mapping.get((column.lower())))
        data_table_dtype_lst.append(data_type_mapping.get((column.lower())))
    summary_statistics_table=pd.DataFrame(list(zip(columns,data_table_object_lst,data_table_dtype_lst,data_table_mean_lst,data_table_median_lst,data_table_min_lst,data_table_max_lst,data_table_std_lst)),
                                      columns=['Variable','Data','Data Type','Mean','Median','Min','Max','Standard Deviation'])
    return summary_statistics_table

In [21]:
summary_statistics_table=statistics_table(data)

In [22]:
#removing means from non continous and discrete variables
summary_statistics_table["Mean"] = summary_statistics_table.apply(lambda row: row["Mean"] if row["Data Type"].lower() in ["continuous", "discrete"] else None, axis=1)
summary_statistics_table["Mean"] = summary_statistics_table.apply(lambda row: row["Mean"] if row["Data Type"].lower() in ["continuous", "discrete"] else None, axis=1)
summary_statistics_table["Median"] = summary_statistics_table.apply(lambda row: None if row["Data Type"].lower() == "nominal" else row["Median"] , axis=1)
summary_statistics_table=summary_statistics_table.merge(missing_values_percent,left_on='Variable',right_index=True)
summary_statistics_table.sort_values(by=['Missing Values %','Variable'],inplace=True)
summary_statistics_table.reset_index(drop=True,inplace=True)

summary_statistics_table_1=summary_statistics_table[['Variable','Data','Data Type']]
summary_statistics_table_1["Description"]=summary_statistics_table_1["Variable"].apply(lambda x: full_variable_name_mapping.get(x.lower()))
summary_statistics_table_1=summary_statistics_table_1[['Variable','Description','Data','Data Type']]
#summary_statistics_table_1.to_excel('Summary Statistics Table_Figure_3.xlsx',index=False)  

summary_statistics_table_2=summary_statistics_table.drop(columns=['Data','Data Type'],axis=1)
#summary_statistics_table_2.to_excel('Summary Statistics Table_Figure_2.xlsx',index=False)  

# Creating a function to generate the statistics table for excel

/var/folders/0f/v6wgxn554tx62wm1nqjqw5hw0000gn/T/ipykernel_3444/1696420439.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_statistics_table_1["Description"]=summary_statistics_table_1["Variable"].apply(lambda x: full_variable_name_mapping.get(x.lower()))


In [23]:
remove_attributes=list(summary_statistics_table[summary_statistics_table_2['Missing Values %']>56.00]['Variable'])
#creating a list of variables with more than 56% missing values

# Dropping the below variables as they are not important
remove_attributes.append('SURVMNTH')
remove_attributes.append('SURVYEAR')
remove_attributes.append('REC_NUM')
remove_attributes.append('FINALWT')

##################################

remove_attributes.append('LFSSTAT')
remove_attributes.append('MJH')
remove_attributes.append('FTPTMAIN')
#deleting because we are only looking at full timer workers not parttime.
remove_attributes.append('PERMTEMP')
#already filtered for full time jobs
remove_attributes.append('SCHOOLN')
#only looking at non students and not full time or part time students

##################################
 
remove_attributes.append('HRSAWAY')
#Hours away from work, part-week absence only low variance has 79 perecent 0s
remove_attributes.append('PAIDOT')
#Paid over time has low variance 83% of zeros
remove_attributes.append('UNPAIDOT')
#Upaid overtime has 83% of zeros
remove_attributes.append('XTRAHRS')
#Contains over 70% of zeroes

data2=data.drop(columns=remove_attributes)
#dropping the variables with more than 56% missing values and creating a new dataframe.


In [24]:
data2['SEX']=data2['SEX'].map({1:0,2:1})
#Male is 0 and Female is 1. 
data2['IMMIG']=data2['IMMIG'].map({1:1,2:1,3:0})
#immigrant is 1 and non immigrant is 0data2['MARSTAT']=data2['MARSTAT'].map({1:1,2:0,3:0,4:0,5:0,6:0})
data2['MARSTAT']=data2['MARSTAT'].map({1:1,2:0,3:0,4:0,5:0,6:0})
#Married is 1 and not married is 0
data2['CMA']=data2['CMA'].map({0:0,1:1,2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1})
#1 is for the 9 largest CMAs and 0 is for the rest.

data2['EFAMTYPE']=data2['EFAMTYPE'].map({
    1:'Person not in an economic family',
    2:'Dual-earner couple, no children or none under 25',
    3:'Dual-earner couple, youngest child 0 to 17',
    4:'Dual-earner couple, youngest child 18 to 24',
    5:'Single-earner couple, male employed, no children or none under 25',
    6:'Single-earner couple, male employed, youngest child 0 to 17',
    7:'Single-earner couple, male employed, youngest child 18 to 24',
    8:'Single-earner couple, female employed, no children or none under 25',
    9:'Single-earner couple, female employed, youngest child 0 to 17',
    10:'Single-earner couple, female employed, youngest child 18 to 24',
    11:'Non-earner couple, no children or none under 25',
    12:'Non-earner couple, youngest child 0 to 17',
    13:'Non-earner couple, youngest child 18 to 24',
    14:'Lone-parent family, parent employed, youngest child 0 to 17',
    15:'Lone-parent family, parent employed, youngest child 18 to 24',
    16:'Lone-parent family, parent not employed, youngest child 0 to 17',
    17:'Lone-parent family, parent not employed, youngest child 18 to 24',
    18:'Other families'
})
# Mapping the economic family type

data2['COWMAIN']=data2['COWMAIN'].map({1:'Public sector employees',2:'Private sector employees'})
# Mapping the class of worker

data2['PROV']=data2['PROV'].map({10:'NL',11:'PE',12:'NS',13:'NB',24:'QC',35:'ON',46:'MB',47:'SK',48:'AB',59:'BC'})
# Mapping the provinces

data2['AGE_12']=data2['AGE_12'].map({
    1:'15-19'
    ,2:'20-24'
    ,3:'25-29'
    ,4:'30-34'
    ,5:'35-39'
    ,6:'40-44'
    ,7:'45-49'
    ,8:'50-54'
    ,9:'55-59'
    ,10:'60-64'
    ,11:'65-69'
    ,12:'70+'})
# Mapping the age groups

data2['EDUC']=data2['EDUC'].map({
    0:'0 to 8 years'
    ,1:'Some high school'
    ,2:'High school graduate'
    ,3:'Some post-secondary'
    ,4:'Post-secondary certificate or diploma'
    ,5:'Bachelor\'s degree'
    ,6:'Above bachelor\'s degree'})
# Mapping the education levels

data2['NAICS_21']=data2['NAICS_21'].map({
1:'Agriculture'
,2:'Forestry and logging and support activities for forestry'
,3:'Fishing, hunting and trapping'
,4:'Mining, quarrying, and oil and gas extraction'
,5:'Utilities'
,6:'Construction'
,7:'Manufacturing - durable goods'
,8:'Manufacturing - non-durable goods'
,9:'Wholesale trade'
,10:'Retail trade'
,11:'Transportation and warehousing'
,12:'Finance and insurance'
,13:'Real estate and rental and leasing'
,14:'Professional, scientific and technical services'
,15:'Business, building and other support services'
,16:'Educational services'
,17:'Health care and social assistance'
,18:'Information, culture and recreation'
,19:'Accommodation and food services'
,20:'Other services (except public administration)'
,21:'Public administration' 
})
# Mapping the industry

data2['FIRMSIZE']=data2['FIRMSIZE'].map({
     1:'Less than 20 employees'
    ,2:'20-99 employees'
    ,3:'100-500 employees'
    ,4:'More than 500 employees'})
# Mapping the firm size

data2['UNION']=data2['UNION'].map({
    1:'Union member',
    2:'Not a member but covered by a union contract or collective agreement',
    3:'Non-unionized'})
# Mapping the union status

data2['NOC_43']=data2['NOC_43'].map({
    1:'Legislative and senior management occupations',
    2:'Specialized middle management occupations',
    3:'Middle management occupations in retail and wholesale trade and customer services',
    4:'Middle management occupations in trades, transportation, production and utilities',
    5:'Professional occupations in finance',
    6:'Professional occupations in business',
    7:'Administrative and financial supervisors and specialized administrative occupations',
    8:'Administrative occupations and transportation logistics occupations',
    9:'Administrative and financial support and supply chain logistics occupations',
    10:'Professional occupations in natural sciences',
    11:'Professional occupations in applied sciences (except engineering)',
    12:'Professional occupations in engineering',
    13:'Technical occupations related to natural and applied sciences',
    14:'Health treating and consultation services professionals',
    15:'Therapy and assessment professionals',
    16:'Nursing and allied health professionals',
    17:'Technical occupations in health',
    18:'Assisting occupations in support of health services',
    19:'Professional occupations in law',
    20:'Professional occupations in education services',
    21:'Professional occupations in social and community services',
    22:'Professional occupations in government services',
    23:'Occupations in front-line public protection services',
    24:'Paraprofessional occupations in legal, social, community and education services',
    25:'Assisting occupations in education and in legal and public protection',
    26:'Care providers and public protection support occupations and student monitors, crossing guards and related occupations',
    27:'Professional occupations in art and culture',
    28:'Technical occupations in art, culture and sport',
    29:'Occupations in art, culture and sport',
    30:'Support occupations in art, culture and sport',
    31:'Retail sales and service supervisors and specialized occupations in sales and services',
    32:'Occupations in sales and services',
    33:'Sales and service representatives and other customer and personal services occupations',
    34:'Sales and service support occupations',
    35:'Technical trades and transportation officers and controllers',
    36:'General trades',
    37:'Mail and message distribution, other transport equipment operators and related maintenance workers',
    38:'Helpers and labourers and other transport drivers, operators and labourers',
    39:'Supervisors and occupations in natural resources, agriculture and related production',
    40:'Workers and labourers in natural resources, agriculture and related production',
    41:'Supervisors, central control and process operators in processing, manufacturing and utilities and aircraft assemblers and inspectors',
    42:'Machine operators, assemblers and inspectors in processing, manufacturing and printing',
    43:'Labourers in processing, manufacturing and utilities'
})
# Mapping the occupation

data2['NOC_10']=data2['NOC_10'].map({
    1:'Management occupations',
    2:'Business, finance and administration occupations, except management',
    3:'Natural and applied sciences and related occupations, except management',
    4:'Health occupations, except management',
    5:'Occupations in education, law and social, community and government services, except management',
    6:'Occupations in art, culture, recreation and sport, except management',
    7:'Sales and service occupations, except management',
    8:'Trades, transport and equipment operators and related occupations, except management',
    9:'Natural resources, agriculture and related production occupations, except management',
    10:'Occupations in manufacturing and utilities, except management'})
# Mapping the occupation


In [ ]:
#profile2 = ProfileReport(data2, title="Profiling Report")
#profile2.to_file("EDA after removing non-correlated features.html")

In [25]:
data3=data2[:]

In [26]:
#will be dropping the remaaining measures becauase of high correlation with other variables
remove_attributes2=[]
# Creating an empty list to store the variables to be removed
remove_attributes2.append('NOC_43')
#correlated with sex
remove_attributes2.append('NOC_10')
#correlated with sex
remove_attributes2.append('ESTSIZE')
#dropping MJH because it only contains one value, so it is not useful for the model
remove_attributes2.append('AHRSMAIN')
#correlated with UTOTHRS, FTPTMAIN, UHRSMAIN, ATOTHRS
remove_attributes2.append('UTOTHRS')
#Usual hours worked per week at all jobs, only looking at single job holder
remove_attributes2.append('ATOTHRS')
#Actual hours worked per week at all jobs, only looking at single job holder
remove_attributes2.append('COWMAIN')
#removing cowmain because it is correlated with naics_21
remove_attributes2.append('EFAMTYPE')
#removing efamtype because it is correlated with marstat

#remove_attributes2.append('UNION')
#removing this because it is correlated tenure

data3=data3.drop(columns=remove_attributes2)



In [ ]:
#profile3 = ProfileReport(data3, title="Profiling Report")
#profile3.to_file("EDA after removing the correlated features.html")

In [27]:
master_list_of_variables_removed=remove_attributes+remove_attributes2
null_attributes_full_variable_name=[]
for attribute in master_list_of_variables_removed:
    null_attributes_full_variable_name.append(full_variable_name_mapping.get(attribute.lower()))

null_attributes_table=pd.DataFrame({'Variable':master_list_of_variables_removed,'Description':null_attributes_full_variable_name})
#null_attributes_table

In [ ]:
#data3.info()

In [29]:
# Feature series to be plotted
feature_series = [data3['HRLYEARN'], data3['UHRSMAIN'], data3['TENURE']]

'''
# Create subplots
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(20, 7))

# Add titles and labels
titles = ['Hourly Earnings (in dollars)', 'Usual Hours Worked (per week)', 'Job Tenure (in months)']
x_labels = ['Dollars', 'Hours', 'Months']


# Plot each feature in a separate subplot
for i, axi in enumerate(ax.flat):
    try:
        axi.hist(feature_series[i], bins=50, color='skyblue', edgecolor='black')
        axi.set_title(titles[i], fontsize=14)
        axi.set_xlabel(x_labels[i], fontsize=12)
        axi.set_ylabel('Frequency', fontsize=12)
    except IndexError:
        break

plt.tight_layout()
#plt.show()
'''

"\n# Create subplots\nfig, ax = plt.subplots(ncols=3, nrows=1, figsize=(20, 7))\n\n# Add titles and labels\ntitles = ['Hourly Earnings (in dollars)', 'Usual Hours Worked (per week)', 'Job Tenure (in months)']\nx_labels = ['Dollars', 'Hours', 'Months']\n\n\n# Plot each feature in a separate subplot\nfor i, axi in enumerate(ax.flat):\n    try:\n        axi.hist(feature_series[i], bins=50, color='skyblue', edgecolor='black')\n        axi.set_title(titles[i], fontsize=14)\n        axi.set_xlabel(x_labels[i], fontsize=12)\n        axi.set_ylabel('Frequency', fontsize=12)\n    except IndexError:\n        break\n\nplt.tight_layout()\n#plt.show()\n"

In [30]:
# Perform Shapiro-Wilk test
for column in ['HRLYEARN','UHRSMAIN','TENURE']:
    stat, p = stats.shapiro(data3[column])
    print(f'Statistic: {stat}, p-value: {p}')
    # Interpret the result
    alpha = 0.05
    if p > alpha:
        print(f'{column} looks like a normal distribution (fail to reject H0)')
    else:
        print(f'{column} does not look like a normal distribution (reject H0)')

Statistic: 0.8565079569816589, p-value: 0.0
HRLYEARN does not look like a normal distribution (reject H0)
Statistic: 0.6125021576881409, p-value: 0.0
UHRSMAIN does not look like a normal distribution (reject H0)
Statistic: 0.8704522252082825, p-value: 0.0
TENURE does not look like a normal distribution (reject H0)


/Users/hk/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [31]:
# Feature series to be plotted
feature_series = [data3['HRLYEARN'], data3['UHRSMAIN'], data3['TENURE']]
feature_labels = ['Hourly Earnings (in dollars)', 'Usual Hours Worked (per week)', 'Job Tenure (in months)']

'''
# Customize flier properties for outliers
flierprops = dict(marker='o', markerfacecolor='r', markersize=12,
                  linestyle='none', markeredgecolor='g')

# Create subplots
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(20, 7))

# Plot each feature in a separate subplot

for i, axi in enumerate(ax.flat):
    try:
        axi.boxplot(feature_series[i], flierprops=flierprops, patch_artist=True,
                    boxprops=dict(facecolor='skyblue', color='black'),
                    capprops=dict(color='black'),
                    whiskerprops=dict(color='black'),
                    medianprops=dict(color='red'))
        axi.set_title(feature_labels[i], fontsize=14)
        axi.tick_params(axis='y', labelsize=12)
        axi.tick_params(axis='x', labelsize=0)
    except IndexError:
        break

plt.tight_layout()
#plt.show()
'''

"\n# Customize flier properties for outliers\nflierprops = dict(marker='o', markerfacecolor='r', markersize=12,\n                  linestyle='none', markeredgecolor='g')\n\n# Create subplots\nfig, ax = plt.subplots(ncols=3, nrows=1, figsize=(20, 7))\n\n# Plot each feature in a separate subplot\n\nfor i, axi in enumerate(ax.flat):\n    try:\n        axi.boxplot(feature_series[i], flierprops=flierprops, patch_artist=True,\n                    boxprops=dict(facecolor='skyblue', color='black'),\n                    capprops=dict(color='black'),\n                    whiskerprops=dict(color='black'),\n                    medianprops=dict(color='red'))\n        axi.set_title(feature_labels[i], fontsize=14)\n        axi.tick_params(axis='y', labelsize=12)\n        axi.tick_params(axis='x', labelsize=0)\n    except IndexError:\n        break\n\nplt.tight_layout()\n#plt.show()\n"

In [32]:
# 5 number summary for HRLYEARN.
Q1_HRLYEARN = np.percentile(data3['HRLYEARN'], 25)
Q2_HRLYEARN  = np.percentile(data3['HRLYEARN'], 50)
Q3_HRLYEARN  = np.percentile(data3['HRLYEARN'], 75)
min_value_HRLYEARN  = np.min(data3['HRLYEARN'])
max_value_HRLYEARN  = np.max(data3['HRLYEARN'])
IQR_HRLYEARN = Q3_HRLYEARN  - Q1_HRLYEARN 
#print("Five Number Summary Q1,Q2,Q3,min_value,max_value, and IQR:",Q1_HRLYEARN ,Q2_HRLYEARN ,Q3_HRLYEARN , min_value_HRLYEARN ,max_value_HRLYEARN , IQR_HRLYEARN)

In [33]:
Q1_UHRSMAIN = np.percentile(data2['UHRSMAIN'], 25)
Q2_UHRSMAIN = np.percentile(data2['UHRSMAIN'], 50)
Q3_UHRSMAIN = np.percentile(data2['UHRSMAIN'], 75)
min_value_UHRSMAIN = np.min(data2['UHRSMAIN'])
max_value_UHRSMAIN = np.max(data2['UHRSMAIN'])
IQR_UHRSMAIN = Q3_UHRSMAIN - Q1_UHRSMAIN
#print("Five Number Summary Q1,Q2,Q3,min_value,max_value, and IQR:",Q1_UHRSMAIN,Q2_UHRSMAIN,Q3_UHRSMAIN, min_value_UHRSMAIN,max_value_UHRSMAIN, IQR_UHRSMAIN)


In [34]:
Q1_TENURE = np.percentile(data2['TENURE'], 25)
Q2_TENURE = np.percentile(data2['TENURE'], 50)
Q3_TENURE = np.percentile(data2['TENURE'], 75)
min_value_TENURE = np.min(data2['TENURE'])
max_value_TENURE = np.max(data2['TENURE'])
IQR_TENURE = Q3_TENURE - Q1_TENURE
#print("Five Number Summary Q1,Q2,Q3,min_value,max_value, and IQR:",Q1_TENURE,Q2_TENURE,Q3_TENURE, min_value_TENURE,max_value_TENURE, IQR_TENURE)


In [35]:
#checking for outliers in the response variable using IQR
outliers_HRLYEARN = data3[(data2['HRLYEARN'] < (Q1_HRLYEARN - 1.5 * IQR_HRLYEARN)) | (data3['HRLYEARN'] > (Q3_HRLYEARN + 1.5 * IQR_HRLYEARN))]
outliers_UHRSMAIN = data3[(data3['UHRSMAIN'] < (Q1_UHRSMAIN - 1.5 * IQR_UHRSMAIN)) | (data3['UHRSMAIN'] > (Q3_UHRSMAIN + 1.5 * IQR_UHRSMAIN))]
outliers_TENURE = data3[(data3['TENURE'] < (Q1_TENURE - 1.5 * IQR_TENURE)) | (data3['TENURE'] > (Q3_TENURE + 1.5 * IQR_TENURE))]
#print("The number of outliers for HRLYEARN, UHRSMAIN, and TENURE are:",len(outliers_HRLYEARN),len(outliers_UHRSMAIN),len(outliers_TENURE))
#print("The shape of the working dataset is:",data3.shape)
#The outliers account for 14.83% of the data. The outliers will be removed from the dataset.

In [36]:
#Creating a dataframe with the all the unique values in data3 and the number of classes in the categorical columns
unique_values_table=pd.DataFrame()
for column in data3.columns:
    unique_values_table[column]=[data3[column].nunique()]
unique_values_table=unique_values_table.T
unique_values_table=unique_values_table.rename(columns={0:'Unique Values'})
unique_values_table=unique_values_table.reset_index()
unique_values_table=unique_values_table.rename(columns={'index':'Variables'})
unique_values_table=unique_values_table.sort_values(by='Unique Values',ascending=False)
unique_values_table.reset_index(drop=True,inplace=True)
#unique_values_table.to_excel('Unique Values Table_Figure_4.xlsx',index=False)

In [37]:
data4=data3[:]
# Crating a newdataframe to remove the outliers from data3

In [38]:
#removing the outliers from the dataset
data4 = data4[(data4['HRLYEARN'] >= (Q1_HRLYEARN - 1.5 * IQR_HRLYEARN)) & (data4['HRLYEARN'] <= (Q3_HRLYEARN + 1.5 * IQR_HRLYEARN))]
data4 = data4[(data4['UHRSMAIN'] >= (Q1_UHRSMAIN - 1.5 * IQR_UHRSMAIN)) & (data4['UHRSMAIN'] <= (Q3_UHRSMAIN + 1.5 * IQR_UHRSMAIN))]
data4 = data4[(data4['TENURE'] >= (Q1_TENURE - 1.5 * IQR_TENURE)) & (data4['TENURE'] <= (Q3_TENURE + 1.5 * IQR_TENURE))]

In [ ]:
#print("The median of the response variable before and after removing the outliers:", data3["HRLYEARN"].median(),data4["HRLYEARN"].median())

In [39]:
sample_data = data4["HRLYEARN"]  
# Hypothetical median
hypothetical_median = 33.55 #Population median
differences = sample_data - hypothetical_median

# Perform one-sample Wilcoxon signed-rank test
statistic, p_value = stats.wilcoxon(differences)
#print(f"Wilcoxon signed-rank test: statistic = {statistic}, p-value = {p_value}")


In [40]:
summary_statistics_table_continuous_variables_before_outliers=statistics_table(data3[['HRLYEARN','UHRSMAIN','TENURE']])
summary_statistics_table_continuous_variables_after_outliers=statistics_table(data4[['HRLYEARN','UHRSMAIN','TENURE']])
# Creating the summary statistics table for the continuous variables before and after removing the outliers

#summary_statistics_table_continuous_variables_before_outliers.to_excel('Continuous Variables Before Removing outliers.xlsx',index=False)  
#summary_statistics_table_continuous_variables_after_outliers.to_excel('Continuous Variables After Removing outliers.xlsx',index=False)  
# Exporting the summary statistics table for the continuous variables before and after removing the outliers




In [41]:
data5=data4[:]
# Creating new data frame after removing the outliers

In [42]:
data5.rename(columns={'AGE_12': 'AGE'}, inplace=True)
# Renaming the age column
data5.rename(columns={'NAICS_21':'NAICS'},inplace=True)

In [43]:
categorical_features = ['PROV', 'CMA', 'AGE', 'SEX', 'MARSTAT', 'EDUC', 'IMMIG', 'NAICS', 'UNION', 'FIRMSIZE']
continuous_features = ['UHRSMAIN', 'TENURE']

# Define the response variable
y = data5['HRLYEARN']

# Define the feature variables
X = data5.drop(columns=['HRLYEARN'])

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), continuous_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Fit and transform the data
X_transformed = preprocessor.fit_transform(X)

# Check the shape of the transformed data
print("Shape of transformed data:", X_transformed.shape)

# Get feature names after transformation
categorical_transformer = preprocessor.named_transformers_['cat']
onehot_feature_names = categorical_transformer.get_feature_names_out(categorical_features)
feature_names = np.append(continuous_features, onehot_feature_names)

# Check the length of feature names
print("Number of feature names:", len(feature_names))

# Convert sparse matrix to dense matrix
X_transformed_dense = X_transformed.toarray()

# Create a DataFrame with transformed features and set the original index
try:
    X_transformed_df = pd.DataFrame(X_transformed_dense, columns=feature_names, index=data5.index)
    print("DataFrame created successfully with transformed features.")
    print(X_transformed_df.head())
except ValueError as e:
    print("Error creating DataFrame:", e)
    print("Number of columns in transformed data:", X_transformed_dense.shape[1])
    print("Number of feature names:", len(feature_names))

Shape of transformed data: (127821, 55)
Number of feature names: 55
DataFrame created successfully with transformed features.
    UHRSMAIN    TENURE  PROV_BC  PROV_MB  PROV_NB  PROV_NL  PROV_NS  PROV_ON  \
6   0.373737  1.000000      0.0      0.0      0.0      0.0      0.0      0.0   
7   0.626263  0.066946      1.0      0.0      0.0      0.0      0.0      0.0   
10  0.121212  1.000000      1.0      0.0      0.0      0.0      0.0      0.0   
11  0.626263  0.000000      0.0      0.0      0.0      0.0      0.0      0.0   
16  0.373737  0.121339      0.0      0.0      0.0      0.0      0.0      1.0   

    PROV_PE  PROV_QC  ...  NAICS_Real estate and rental and leasing  \
6       0.0      1.0  ...                                       0.0   
7       0.0      0.0  ...                                       0.0   
10      0.0      0.0  ...                                       0.0   
11      0.0      1.0  ...                                       0.0   
16      0.0      0.0  ...             

In [44]:
# Define the Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'model__n_estimators': [10],
    'model__max_features': [1.0],
    'model__max_depth': [None],
    'model__min_samples_split': [2],
    'model__min_samples_leaf': [1],
    'model__bootstrap': [True]
}
# Define the KFold cross-validator
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Define RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,n_iter=1,cv=kf,random_state=42, n_jobs=-1)

# Can only do 1 iteration because of computaional constraints. 

In [45]:
# Fit the RandomizedSearchCV
random_search.fit(X, y)

# Get the best model from RandomizedSearchCV
best_model = random_search.best_estimator_

# Print the best hyperparameters
#print(f"Best hyperparameters: {random_search.best_params_}")

In [46]:
# Retreived the best model after fitting the RandomizedSearchCV
model = best_model.named_steps['model']

# Retreiving the fature names
categorical_transformer = best_model.named_steps['preprocessor'].named_transformers_['cat']
onehot_feature_names = categorical_transformer.get_feature_names_out(categorical_features)
feature_names = np.append(continuous_features, onehot_feature_names)

# Extracted feature importances
feature_importances = model.feature_importances_

# Created a DataFrame for better visualization
features_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sorted the DataFrame by importance
features_df = features_df.sort_values(by='Importance', ascending=False)
# Kept 95% of the cumulative importance

# Calculate cumulative importance
features_df['Cumulative Importance'] = features_df['Importance'].cumsum()

#features_df.to_excel('Feature Importance Table.xlsx',index=False)
# Exporting the feature importance table


# Display the DataFrame with importance
#print(features_df)
'''
# Plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=features_df, palette='viridis')
plt.title('Feature Importances from Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()
'''


"\n# Plot feature importances\nplt.figure(figsize=(12, 8))\nsns.barplot(x='Importance', y='Feature', data=features_df, palette='viridis')\nplt.title('Feature Importances from Random Forest')\nplt.xlabel('Importance')\nplt.ylabel('Feature')\nplt.show()\n"

In [47]:
# Set the cumulative importance threshold
threshold = 0.95

# Identify the features to keep
features_to_keep = features_df[features_df['Cumulative Importance'] <= threshold]['Feature']

#print(f"Number of features to keep: {len(features_to_keep)}")
#print(f"Features to keep:\n{features_to_keep}")

# Subset the transformed dataset to include only the top 10 features
X_reduced = X_transformed_df[features_to_keep]
#print("Reduced dataset shape:", X_reduced.shape)

In [ ]:
#print(X_reduced.shape)
#print(data5['HRLYEARN'].shape) 

In [48]:
#This is after the linearity assumption is done
preparing_features_for_models=X_reduced.copy()

In [49]:
# Ensure indices are aligned
X_reduced = X_reduced.reset_index(drop=True)
y = data5['HRLYEARN'].reset_index(drop=True)

In [159]:
print(X_reduced.shape)
print(y.shape) 

(127821, 41)
(127821,)


In [ ]:
''''
# Define the independent variables (X)
X = X_reduced

# Add a constant to the model
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

model_summary = model.summary()
#print(model_summary)

# Get the residuals
residuals = model.resid

# 1. Check for independence (Durbin-Watson test)
dw_test = sm.stats.stattools.durbin_watson(residuals)
#print('Durbin-Watson test statistic:', dw_test)

# 2. Check for normality (Q-Q plot and Shapiro-Wilk test)
plt.figure(figsize=(8, 6))
plt.hist(residuals, color='skyblue', edgecolor='black')
plt.title('Histogram of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

shapiro_test = stats.shapiro(residuals)
#print('Shapiro-Wilk test statistic:', shapiro_test)

# 3. Check for mean of residuals equal to zero
mean_residuals = residuals.mean()
#print('Mean of residuals:', mean_residuals)

# 4. Check for homoscedasticity (constant variance)
plt.scatter(model.fittedvalues, residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Fitted values vs Residuals')
plt.show()
'''

In [ ]:
'''
# Apply log transformation to the dependent variable to fix the normality issue and check the residuals again
# This fixed the normaility issue but not the homoscedasticity issue
 
#transformed dependent variable (y)
log_y=np.log(y)

# Add a constant to the model
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(log_y, X).fit()

# Get the residuals
residuals = model.resid

# 1. Check for independence (Durbin-Watson test)
dw_test = sm.stats.stattools.durbin_watson(residuals)
#print('Durbin-Watson test statistic:', dw_test)

# 2. Check for normality (Q-Q plot and Shapiro-Wilk test)
plt.figure(figsize=(8, 6))
plt.hist(residuals, color='skyblue', edgecolor='black')
plt.title('Histogram of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

shapiro_test = stats.shapiro(residuals)
#print('Shapiro-Wilk test statistic:', shapiro_test)

# 3. Check for mean of residuals equal to zero
mean_residuals = residuals.mean()
#print('Mean of residuals:', mean_residuals)

# 4. Check for homoscedasticity (constant variance)
plt.scatter(model.fittedvalues, residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Fitted values vs Residuals')
plt.show()
'''

In [ ]:
'''
# Useing weighted least squares (WLS) to account for heteroscedasticity, but does not workw well.
model = sm.OLS(y, X).fit()
weights = 1 / model.fittedvalues
model_wls = sm.WLS(y, X, weights=weights).fit()

# Check residuals for homoscedasticity
residuals_wls = model_wls.resid
plt.scatter(model_wls.fittedvalues, residuals_wls)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Fitted values vs Residuals (WLS)')
plt.show()
'''

In [50]:
features= preparing_features_for_models.copy()
features.drop(columns=['TENURE','UHRSMAIN'],inplace=True)
#dropping these features because they are scaled, and we need to keep the original values for the non-linear models, and scaled later.
continuous_features=data5[['TENURE', 'UHRSMAIN']]

#features.to_excel('Features .xlsx',index=True)
#continuous_features.to_excel('Continuous Features.xlsx',index=True)
#data5['HRLYEARN'].to_excel('Response Variable.xlsx',index=True)

#features=features.reset_index(drop=True)
#continuous_features=continuous_features.reset_index(drop=True)

#data5.to_excel('Data5.xlsx',index=True)

features=features.merge(continuous_features,left_index=True,right_index=True)

print(features.shape)
print(continuous_features.shape)

label = data5['HRLYEARN']
#These will be used for the non-linear machine learning algoirthms. 
label=label.reset_index(drop=True)



(127821, 41)
(127821, 2)


In [52]:
all_the_feautres_without_rf=X_transformed_df.copy()
#This has all the feautures, not dropping anything

all_the_feautres_without_rf.drop(columns=['TENURE','UHRSMAIN'],inplace=True)
#dropping these features because they are scaled, and we need to keep the original values for the non-linear models, and scaled later.

all_the_feautres_without_rf=all_the_feautres_without_rf.merge(continuous_features,left_index=True,right_index=True)

In [53]:
top_features=features.copy()
#This is dropping the imbalances detected after hot ones encoding 

top_features.drop(columns=[
'NAICS_Public administration',
'NAICS_Mining, quarrying, and oil and gas extraction',
'NAICS_Professional, scientific and technical services',
'NAICS_Retail trade',
'AGE_60-64',
'PROV_MB',
'NAICS_Finance and insurance',
'AGE_25-29',
'PROV_SK',
'NAICS_Construction',
'NAICS_Utilities',
'PROV_NS',
'NAICS_Manufacturing - non-durable goods',
'NAICS_Wholesale trade',
'PROV_NB',
'AGE_20-24',
'NAICS_Manufacturing - durable goods',
],axis=1,inplace=True)



In [ ]:
''''
profile4 = ProfileReport(top_features_and_label_for_profile_report, title="Profiling Report")
profile4.to_file("EDA of the top features.html")
#Creating the final profile report
'''

In [ ]:
# Concatenate the DataFrame and the Series
data_aprori = features.copy()
data_aprori.drop(columns=['TENURE','UHRSMAIN'],inplace=True)
#Need to drop TENURE and UHRSMAIN because they scaled and transformed, but will need the original values in order to classify them below into 5 buckets.
data_aprori.info()

In [ ]:
numeric_data=data4[['TENURE','UHRSMAIN','HRLYEARN']]
numeric_data.info()

In [ ]:
# List of numerical features to be discretized
numerical_variables = ['UHRSMAIN', 'TENURE','HRLYEARN']
# Initialize the KBinsDiscretizer with 'uniform' strategy for equal-width binning
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
# Fit and transform the data
numeric_data[numerical_variables] = discretizer.fit_transform(numeric_data[numerical_variables])

In [ ]:
# Select the columns to one-hot encode
to_encode = numeric_data[numerical_variables]

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the selected columns
encoded_features = encoder.fit_transform(to_encode)

# Get the feature names after one-hot encoding
encoded_feature_names = encoder.get_feature_names_out(numerical_variables)

# Convert the result back to a DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)

#print("One-Hot Encoded DataFrame:\n", encoded_df.head())

In [ ]:
data_aprori = pd.concat([data_aprori, encoded_df], axis=1)
#add back the one hot encoded variables to the data_aprori dataframe

In [ ]:
data_aprori.info()

In [ ]:
# Get the bin edges
bin_edges = discretizer.bin_edges_

# Create a dictionary to map feature names to their bin edges
bin_edges_dict = {feature: edges for feature, edges in zip(numerical_variables, bin_edges)}

# Print bin edges for each feature
for feature, edges in bin_edges_dict.items():
    print(f"Bin edges for {feature}: {edges}")

    # Prepare data for DataFrame
bin_info = []

for feature, edges in bin_edges_dict.items():
    for i in range(len(edges) - 1):
        bin_info.append({
            'Feature': feature,
            'Bin Index': i,
            'Bin Start': edges[i],
            'Bin End': edges[i + 1]
        })

# Create DataFrame from bin information
bin_info_df = pd.DataFrame(bin_info)

# Display the DataFrame
print(bin_info_df)

#bin_info_df.to_excel('Binning Information Table.xlsx',index=False)

In [ ]:
# Apply the Apriori algorithm
frequent_itemsets = apriori(data_aprori, min_support=0.1, use_colnames=True)
print("Frequent itemsets:\n", frequent_itemsets.head())

# Extract association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
#rint("Association rules:\n", rules.head())

# Define the desired RHS items
desired_rhs = ['HRLYEARN_0.0', 'HRLYEARN_1.0', 'HRLYEARN_2.0', 'HRLYEARN_3.0', 'HRLYEARN_4.0']

# Filter rules by RHS
filtered_rules = rules[rules['consequents'].apply(lambda x: any(item in x for item in desired_rhs))]

print("Filtered association rules:\n", filtered_rules)



In [ ]:
# Generate a report with the important rules
report = filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

# Save the report to a CSV file
#report.to_csv('important_association_rules_hrlyearn.csv', index=False)

print("Report generated: important_association_rules_hrlyearn.csv")

In [67]:
continuous_features = ['UHRSMAIN', 'TENURE']
#Defining the continuous features

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), continuous_features)
    ])
#Setting up the column transformer

kf = KFold(n_splits=10, shuffle=True, random_state=42)
#Initializing K-Folds

param_grids = {
    'XGB': {
        'model__n_estimators': [1000],
        'model__learning_rate': [0.05],
        'model__max_depth': [3, 4, 5],
        'model__subsample': [0.7, 0.8, 0.9],
        'model__colsample_bytree': [0.7, 0.8, 0.9],
        'model__gamma': [0, 1, 5],
        'model__reg_alpha': [0, 1, 5]
    },
    #'DT': {
    #    'model__criterion': ['mse'],
    #    'model__splitter': ['best', 'random'],
    #    'model__max_depth': [3, 4, 5],
    #    'model__min_samples_split': [2, 3, 4],
    #    'model__min_samples_leaf': [1, 2, 3]
    #},
    'RF': {
        'model__n_estimators': [1000],
        'model__max_depth': [3, 4, 5],
        'model__min_samples_split': [2, 3, 4],
        'model__min_samples_leaf': [1, 2, 3]
    },
    'SVR': {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__degree': [2, 3, 4, 5],
        'model__gamma': ['scale', 'auto'],
        'model__C': [0.1, 1, 10, 100, 1000]
    },
    'KNN': {
        'model__n_neighbors': [3, 5, 7, 9],
        'model__weights': ['uniform', 'distance'],
        'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'model__p': [1, 2]
    }
}
#Defining the parameter grids for each model


models = {
    'XGB': XGBRegressor()
    ,
    #'DT': DecisionTreeRegressor(random_state=42),
    'RF': RandomForestRegressor(random_state=42),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor()
}
# Defining each model

# Perform RandomizedSearchCV for each model
scores = []
#creating an empty list to store the scores

for model_name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    
    grid_search = RandomizedSearchCV(pipeline, param_distributions=param_grids[model_name], n_iter=1, cv=kf, random_state=42, scoring='r2')
    grid_result = grid_search.fit(features, label)
    
    scores.append({
        'model': model_name,
        'best_score': grid_result.best_score_,
        'best_params': grid_result.best_params_
    })
#Running the randomized search for each model

scores_df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
# Creating a datafreame for all the scores
print(scores_df)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import r2_score

# Example: Assuming features and label are already defined
categorical_features = ['PROV', 'CMA', 'AGE', 'SEX', 'MARSTAT', 'EDUC', 'IMMIG', 'NAICS', 'UNION', 'FIRMSIZE']
continuous_features = ['UHRSMAIN', 'TENURE']

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), continuous_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Fit and transform the data
X_transformed = preprocessor.fit_transform(features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, label, test_size=0.2, random_state=42)

# Convert the data to numpy arrays
X_train = X_train.toarray() if hasattr(X_train, 'toarray') else X_train
X_test = X_test.toarray() if hasattr(X_test, 'toarray') else X_test
y_train = np.array(y_train)
y_test = np.array(y_test)

# Define the ANN model
def build_ann(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    return model

# Get the input dimension
input_dim = X_train.shape[1]

# Build the model
model = build_ann(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
mse = model.evaluate(X_test, y_test, verbose=0)
print(f'Mean Squared Error on test data: {mse}')

# Predict on the test data
y_pred = model.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')